In [1]:
from lightautoml_gpu.dataset.seq_np_pd_dataset import SeqNumpyPandasDataset
from lightautoml_gpu.reader.seq import IDSInd, TopInd
from lightautoml_gpu.reader.base import DictToPandasSeqReader
from lightautoml_gpu.tasks import Task
import pandas as pd
import numpy as np
from lightautoml_gpu.automl.base import AutoML
from lightautoml_gpu.ml_algo.boost_lgbm import BoostLGBM
from lightautoml_gpu.ml_algo.boost_cb import BoostCB

from lightautoml_gpu.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBSeqSimpleFeatures, LGBMultiSeqSimpleFeatures
from lightautoml_gpu.pipelines.ml.base import MLPipeline

from lightautoml_gpu.automl.gpu.blend_gpu import WeightedBlenderGPU
from lightautoml_gpu.automl.blend import WeightedBlender

import numpy as np

In [2]:
import pandas as pd
import numpy as np

DATA_FOLDER = '../../data/small_new/relational_data'

fulfilment_center_info = pd.read_csv(f'{DATA_FOLDER}/fulfilment_center_info.csv')
meal_info = pd.read_csv(f'{DATA_FOLDER}/meal_info.csv')
train = pd.read_csv(f'{DATA_FOLDER}/train.csv')

val = 0.05


In [3]:
seq_params = {
              'fulfilment_center_info':{'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'center_id', 'to_id': 'center_id'},
                          },
              'meal_info':{'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'meal_id', 'to_id': 'meal_id'},
                          },
              }
seq_data = {'fulfilment_center_info': fulfilment_center_info[['center_id',
                                                              'city_code',
                                                              'region_code',
                                                              'op_area']],
       'meal_info': meal_info              
       }

X_train = {'plain':train.sample(frac=0.05).reset_index(drop=True) , 
           'seq': seq_data
          }

task = Task('reg', metric='mae')

roles={'target': 'num_orders'}




In [4]:
task = Task('reg', metric='mae')
roles={'target': 'num_orders'}
reader = DictToPandasSeqReader(task=task, seq_params=seq_params)

feats = LGBMultiSeqSimpleFeatures()
model = BoostLGBM()
pipeline_lvl1 = MLPipeline([BoostLGBM(), BoostCB()], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False, blender=WeightedBlender())

In [5]:
%%time

oof_pred = automl.fit_predict(X_train, roles=roles, verbose=3)


[13:03:31] Feats was rejected during automatic roles guess: []
[13:03:31] Layer 1 train process start. Time left 9999999996.28 secs
[13:03:31] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[13:03:31] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[13:03:31] Training until validation scores don't improve for 100 rounds
[13:03:32] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[13:03:32] Training until validation scores don't improve for 100 rounds
[13:03:33] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[13:03:33] Training until validation scores don't improve for 100 rounds
[13:03:34] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[13:03:34] Training until validation scores don't improve for 100 rounds
[13:03:35] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[13:03:35] Training until validation scores don't improve for 100 rounds
[13:03:36] Fitting Lvl_0_Pipe_0_Mod_0_Lig

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[13:03:38] bestTest = 135.7178128
[13:03:38] bestIteration = 2999
[13:03:38] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_CatBoost =====
[13:03:38] 0:	learn: 224.5952860	test: 229.4035389	best: 229.4035389 (0)	total: 1.16ms	remaining: 3.48s


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[13:03:41] bestTest = 132.8503553
[13:03:41] bestIteration = 2997
[13:03:41] Shrink model to first 2998 iterations.
[13:03:41] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_1_CatBoost =====
[13:03:41] 0:	learn: 227.9923933	test: 219.5577247	best: 219.5577247 (0)	total: 959us	remaining: 2.88s


...

[13:03:43] Stopped by overfitting detector  (100 iterations wait)
[13:03:43] bestTest = 126.968793
[13:03:43] bestIteration = 2759
[13:03:43] Shrink model to first 2760 iterations.
[13:03:43] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_1_CatBoost =====
[13:03:43] 0:	learn: 225.1695284	test: 225.3056758	best: 225.3056758 (0)	total: 827us	remaining: 2.48s


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[13:03:45] bestTest = 133.6852218
[13:03:45] bestIteration = 2994
[13:03:45] Shrink model to first 2995 iterations.
[13:03:45] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_1_CatBoost =====
[13:03:45] 0:	learn: 225.2066221	test: 226.1514878	best: 226.1514878 (0)	total: 1.05ms	remaining: 3.15s


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[13:03:47] bestTest = 132.4022718
[13:03:47] bestIteration = 2975
[13:03:47] Shrink model to first 2976 iterations.
[13:03:47] Fitting Lvl_0_Pipe_0_Mod_1_CatBoost finished. score = -132.32506360388444
[13:03:47] Lvl_0_Pipe_0_Mod_1_CatBoost fitting and predicting completed
[13:03:47] Time left 9999999980.09 secs

[13:03:47] Layer 1 training completed.

[13:03:47] Blending: optimization starts with equal weights and score -114.18369205569753
[13:03:47] Blending: iteration 0: score = -107.39745783308815, weights = [0.9334995  0.06650051]
[13:03:47] Blending: iteration 1: score = -107.39745783308815, weights = [0.9334995  0.06650051]
[13:03:47] Blending: no score update. Terminated

CPU times: user 1min, sys: 1.92 s, total: 1min 2s
Wall time: 19.9 s


In [6]:
from lightautoml_gpu.dataset.gpu.gpu_dataset import SeqCudfDataset
from lightautoml_gpu.dataset.gpu.gpu_dataset import SeqDaskCudfDataset
from lightautoml_gpu.reader.gpu.seq_gpu import IDSIndGPU, TopIndGPU
from lightautoml_gpu.reader.gpu.seq_reader_gpu import DictToCudfSeqReader
from lightautoml_gpu.reader.gpu.seq_reader_gpu import DictToDaskCudfSeqReader

from lightautoml_gpu.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml_gpu.ml_algo.gpu.boost_cb_gpu import BoostCBGPU

from lightautoml_gpu.pipelines.features.gpu.lgb_pipeline_gpu import LGBMultiSeqSimpleFeaturesGPU

import cudf
import dask_cudf
import cupy as cp

In [7]:
task = Task('reg', metric='mae', device='gpu')
reader = DictToCudfSeqReader(task=task, seq_params=seq_params, n_jobs=1)

feats = LGBMultiSeqSimpleFeaturesGPU()
model = BoostCBGPU(gpu_ids=[0])
pipeline_lvl1 = MLPipeline([BoostXGB(), BoostCBGPU(gpu_ids=[0])], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False, blender=WeightedBlenderGPU())

In [8]:
oof_pred_gpu = automl.fit_predict(X_train, roles=roles, verbose=3)

[13:03:49] Feats was rejected during automatic roles guess: []
[13:03:49] Layer 1 train process start. Time left 9999999999.39 secs
[13:03:50] Start fitting Lvl_0_Pipe_0_Mod_0_XGB ...
[13:03:50] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[13:04:04] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[13:04:14] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[13:04:24] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[13:04:43] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[13:04:53] Fitting Lvl_0_Pipe_0_Mod_0_XGB finished. score = -105.07485326356658
[13:04:53] Lvl_0_Pipe_0_Mod_0_XGB fitting and predicting completed
[13:04:53] Start fitting Lvl_0_Pipe_0_Mod_1_CatBoostGPU ...
[13:04:53] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[13:04:58] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[13:05:02] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


...

[13:05:07] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[13:05:12] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[13:05:17] Fitting Lvl_0_Pipe_0_Mod_1_CatBoostGPU finished. score = -131.18946181867702
[13:05:17] Lvl_0_Pipe_0_Mod_1_CatBoostGPU fitting and predicting completed
[13:05:17] Time left 9999999912.01 secs

[13:05:17] Layer 1 training completed.

[13:05:17] Blending: Optimization starts with equal weights and score -111.08268116786049
[13:05:17] Blending, iter 0: score = -104.7713256954772, weights = [0.91584045 0.08415958]
[13:05:17] Blending, iter 1: score = -104.7713256954772, weights = [0.91584045 0.08415958]
[13:05:17] No score update. Terminated


In [9]:
gpu_inf = automl.predict(X_train)

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-

In [10]:
automl.to_cpu()

In [11]:
cpu_inf = automl.predict(X_train)

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-

In [12]:
task = Task('reg', metric='mae', device='mgpu')
reader = DictToDaskCudfSeqReader(task=task, seq_params=seq_params, n_jobs=1)

feats = LGBMultiSeqSimpleFeaturesGPU()
model = BoostCBGPU(gpu_ids=[0])
pipeline_lvl1 = MLPipeline([BoostXGB(), BoostCBGPU(gpu_ids=[0])], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False, blender = WeightedBlenderGPU())

In [13]:
oof_pred_mgpu = automl.fit_predict(X_train, roles=roles, verbose=3)

[13:05:20] Feats was rejected during automatic roles guess: []
[13:05:21] Layer 1 train process start. Time left 9999999998.80 secs


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/dask/dataframe/core.py:4948: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/dask/dataframe/core.py:4948: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


[13:05:22] Start fitting Lvl_0_Pipe_0_Mod_0_XGB ...
[13:05:22] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[13:05:36] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[13:05:46] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[13:05:57] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[13:06:16] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_XGB (orig) =====
[13:06:26] Fitting Lvl_0_Pipe_0_Mod_0_XGB finished. score = -105.07485326356658
[13:06:26] Lvl_0_Pipe_0_Mod_0_XGB fitting and predicting completed
[13:06:26] Start fitting Lvl_0_Pipe_0_Mod_1_CatBoostGPU ...
[13:06:26] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[13:06:31] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


...

[13:06:41] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[13:06:46] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_1_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[13:06:51] Fitting Lvl_0_Pipe_0_Mod_1_CatBoostGPU finished. score = -131.18946181867702
[13:06:51] Lvl_0_Pipe_0_Mod_1_CatBoostGPU fitting and predicting completed
[13:06:51] Time left 9999999908.32 secs

[13:06:51] Layer 1 training completed.

[13:06:51] Blending: Optimization starts with equal weights and score -111.08268116786049
[13:06:51] Blending, iter 0: score = -104.7713256954772, weights = [0.91584045 0.08415958]
[13:06:51] Blending, iter 1: score = -104.7713256954772, weights = [0.91584045 0.08415958]
[13:06:51] No score update. Terminated


In [14]:
print(cpu_inf.data.T)
print(gpu_inf.data.T)
print(oof_pred_gpu.data.T)
print(oof_pred.data.T)
print(oof_pred_mgpu.data.T)

[[423.5747    50.790325  43.70079  ... 113.28612  204.9415   127.35535 ]]
[[424.98767   49.410744  43.590008 ...  79.477394 204.9415   119.236984]]
[[360.36606   95.90548   80.62511  ...  44.836693 230.78961   92.04822 ]]
[[439.08554   63.420597 103.57893  ...  41.17908  246.08334   84.58303 ]]
[[360.36606   95.90548   80.62511  ...  44.836693 230.78961   92.04822 ]]
